In [1]:
import json
from json import JSONEncoder
import re
import time
import datetime

In [2]:
class SensorData:
    def __init__(self):
        self.date_time = '' # day for which data is stored
        self.sensor_value = {} # dictionary of parameter and List of values

In [3]:
class MyEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__ 
# http://stackoverflow.com/questions/3768895/how-to-make-a-class-json-serializable    

In [4]:
def parse_sensor_data_test(file_name):
    start_time = time.time()    
    
    PATTERN_HEADER = r'(?m)^(?P<year>\d{4})(?P<month>\d{2})(?P<day>\d{2})'
    PATTERN_DATA = r'(?m)(?P<temperature>\d{4}|NNNN)(?P<humdity>\d{3}|NNN)'
    
    sensor_data_regex = re.compile(PATTERN_DATA)
    
    all_data = []
    with open(file_name,'r', encoding='utf-8') as rdr:            
        for line in rdr:
            # One row contains a day worth of data
            sensor_data = SensorData()
            parameter_names = sensor_data_regex.groupindex.keys()
            for parameter in parameter_names:
                sensor_data.sensor_value[parameter] = []

            match = re.search(PATTERN_HEADER, line)

            if match:
                sensor_data.date_time = \
                    '-'.join([match.group('year'),match.group('month'),match.group('day')])
                
                header_len = len(match.group(0))
                
                match_iter = sensor_data_regex.finditer(line,header_len)
                
                for match in match_iter:
                    for parameter in parameter_names:
                        value = match.group(parameter)
                        try:
                            sensor_data.sensor_value[parameter].append(int(value))
                        except:
                            # Handle NNN, NNNN => None
                            sensor_data.sensor_value[parameter].append(None)

                all_data.append(sensor_data)
        
        print ('Elapsed Time For Parsing: {0:.2f}s'.format(time.time()-start_time))
        
        with open(file_name+'.json','w', encoding='utf-8') as wr:
            json.dump(all_data, wr, ensure_ascii=False, cls=MyEncoder, indent=True)
            
        print ('Elapsed Time After Write to File : {0:.2f}s'.format(time.time()-start_time))

In [5]:
file_name =  r"..\Data\SensorData\sensordata_365.txt"
parse_sensor_data_test(file_name)

Elapsed Time For Parsing: 1.39s
Elapsed Time After Write to File : 3.39s
